In [2]:
import nest_asyncio
import asyncio
import time
import threading

nest_asyncio.apply()
async def fn(p):
  for i in range(5):
    print(i)
    await asyncio.sleep(5)
  print("done")

def thread_driver(p):
    loop = asyncio.new_event_loop()
    #loop = asyncio.get_event_loop()
    loop.run_until_complete(fn(p))


thread_driver(2)

0
1
2
3
4
done


In [ ]:
# Flask web server, can handle concurrent requests. 

In [ ]:
from flask import Flask, request
import time
import asyncio
#from collections import defaultdict
#metric_data = defaultdict(list)
metric_data = []

app = Flask(__name__)

def timethis(theFunction):
    def toTimeWrapper(*args, **kwargs):
          start = time.time()
          result = theFunction(*args, **kwargs)
          end = time.time()
          #metric_data[name].append(
          metric_data.append({"start": start, "end": end, "dt": end - start});
          print(f"processed {len(metric_data)} requests")

          return result
    return toTimeWrapper


@app.route('/add', methods=['GET'])
@timethis
def add():
    number = 99
    time.sleep(5)
    return str(number)

if __name__ == '__main__':
    #app.run(debug=True)
    app.run(debug=False)

In [ ]:
# concurrent http clients using asyncio , much faster than sequencial calls

In [ ]:
import nest_asyncio
import asyncio
import random
import string

import aiohttp

nest_asyncio.apply()

def generate_urls(base_url, num_urls):
    for i in range(num_urls):
        yield base_url + "".join(random.sample(string.ascii_lowercase, 10))


def chunked_http_client(num_chunks):
    """
    Returns a function that can fetch from a URL, ensuring that only
    "num_chunks" of simultaneous connects are made.
    """
    semaphore = asyncio.Semaphore(num_chunks)  # <1>

    async def http_get(url, client_session):  # <2>
        nonlocal semaphore
        async with semaphore:
            async with client_session.request("GET", url) as response:
                return await response.content.read()

    return http_get


async def run_experiment(base_url, num_iter=1000):
    urls = generate_urls(base_url, num_iter)
    http_client = chunked_http_client(100)
    responses_sum = 0
    async with aiohttp.ClientSession() as client_session:
        tasks = [http_client(url, client_session) for url in urls]  # <3>
        for future in asyncio.as_completed(tasks):  # <4>
            data = await future
            responses_sum += len(data)
    return responses_sum


if __name__ == "__main__":
    import time

    loop = asyncio.get_event_loop()
    delay = 100
    num_iter = 1000

    start = time.time()
    result = loop.run_until_complete(
        #run_experiment("http://127.0.0.1:5001/add")
        run_experiment(f"http://127.0.0.1:5005/add2?name=asyncio&delay={delay}&", num_iter))
        
    
    end = time.time()
    print(f"Result: {result}, Time: {end - start}")

In [ ]:
# tornado web server
# similiar performance as Flask web server

In [ ]:
import json
import time
from collections import defaultdict
import nest_asyncio

from tornado import gen, httpserver, ioloop, options, web


nest_asyncio.apply()
class AddMetric(web.RequestHandler):
    metric_data = defaultdict(list)

    async def get(self):
        if self.get_argument("flush", False):
            json.dump(self.metric_data, open("metric_data.json", "w+"))
        else:
            name = self.get_argument("name", "asyncio")
            try:
                delay = int(self.get_argument("delay", 1024))
            except ValueError:
                raise web.HTTPError(400, reason="Invalid value for delay")

            start = time.time()
            #await gen.sleep(delay / 1000.0)
            await gen.sleep(5)
            self.write("42")
            self.finish()
            end = time.time()
            self.metric_data[name].append(
                {"start": start, "end": end, "dt": end - start}
            )


if __name__ == "__main__":
    #options.parse_command_line()
    #port = options.options.port
    port = 5005
    application = web.Application([(r"/add2", AddMetric)])

    http_server = httpserver.HTTPServer(application)
    http_server.listen(port)
    print(("Listening on port: {}".format(port)))
    ioloop.IOLoop.instance().start()

Listening on port: 5005
